In [ ]:
import pickle

In [ ]:
# Load /share/u/can/sae_eval/dictionary_learning/dictionaries/pythia70m_sweep0711/resid_post_layer_4/trainer_10/node_effects.pkl
file_name = '/share/u/can/sae_eval/dictionary_learning/dictionaries/pythia70m_sweep0711/resid_post_layer_4/trainer_10/node_effects.pkl'
with open(file_name, 'rb') as f:
    node_effects = pickle.load(f)

In [ ]:
# class_idxs = list(node_effects.keys())
# node_effects = list(node_effects[class_idxs[0]].values())[0]



# mask = node_effects > T_inscope_effect
# node_effects

In [ ]:
T_inscope_effect = 1e-8
T_outscope_effect = 5e-3

dict_size = 8192

In [ ]:
from bib_intervention import select_unique_class_features

unique_feats= select_unique_class_features(
    node_effects,
    dict_size,
    T_effect=T_inscope_effect,
    T_max_sideeffect=T_outscope_effect,
    verbose=True,
)

In [ ]:
class_0_feats = unique_feats[0]
class_1_feats = unique_feats[1]
class_0_feats = list(class_0_feats.values())[0]
class_1_feats = list(class_1_feats.values())[0]
class_0_feats.nonzero().flatten().shape

In [ ]:
class_0_nonzero = class_0_feats.nonzero().flatten()
class_1_nonzero = class_1_feats.nonzero().flatten()
(class_0_feats & class_1_feats).sum()
